In [18]:
from keras.models import Model
from keras.layers import LSTM,Input,Dense
import numpy as np

In [2]:
batch_size=64
epochs = 100
latent_dimension = 256
no_samples = 10000
data = 'fra.txt'

In [6]:
#vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(no_samples,len(lines) - 1)]:
    input_text,target_text,_ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [7]:
input_characters

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [8]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f'}

In [9]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Fire!',
 'Help!',
 'Hide.',
 'Hide.',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Sorry?',
 'Attack!',
 'Attack!',
 'Attack!',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Get up.',
 'Get up.',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it!',
 'Got it?',
 '

In [10]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 '\tAha.\n',
 "\tJ'essaye.\n",
 "\tJ'

In [11]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoders_token = len(input_characters)
num_decoder_token = len(target_characters)

In [12]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [13]:
print("No of unique input tokens:",num_encoders_token)
print("No of unique output tokens:",num_decoder_token)
print("Maximum sequence length of input:",max_encoder_seq_length)
print("Maximum sequence length of output:",max_decoder_seq_length)

No of unique input tokens: 71
No of unique output tokens: 92
Maximum sequence length of input: 15
Maximum sequence length of output: 59


In [14]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
output_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [15]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [16]:
output_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 '\xa0': 70,
 '«': 71,
 '»': 72,
 'À': 73,
 'Ç': 74,
 'É': 75,
 'Ê': 76,
 'à': 77,
 'â': 78,
 'ç': 79,
 'è': 80,
 'é': 81,
 'ê': 82,
 'î': 83,
 'ï': 84,
 'ô': 85,
 'ù': 86,
 'û': 87,
 'œ': 88,
 '\u2009': 89,
 '’': 90,
 '\u202f': 91}

In [19]:
#one hot representation
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoders_token),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_token),dtype='float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_token),dtype='float32')

In [20]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,output_token_index[char]] = 1
        if t > 0:
            decoder_target_data[i,t-1,output_token_index[char]] = 1
    decoder_input_data[i,t+1:,output_token_index[' ']] = 1
    decoder_target_data[i,t:,output_token_index[' ']] = 1

In [21]:
encoder_input_data[0].shape

(15, 71)

In [23]:
#defining input_sequence
encoder_inputs = Input(shape=(None,num_encoders_token))
encoders = LSTM(latent_dimension,return_state=True)
encoder_output,state_h,state_c = encoders(encoder_inputs)
encoder_states = [state_h,state_c]

In [26]:
decoder_inputs = Input(shape=(None,num_decoder_token))
decoders = LSTM(latent_dimension,return_sequences=True,return_state=True)
decoder_output,_,_ = decoders(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(num_decoder_token,activation='softmax')
decoder_output = decoder_dense(decoder_output)

In [27]:
model = Model([encoder_inputs,decoder_inputs],decoder_output)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 40s 5ms/step - loss: 1.1447 - accuracy: 0.7331 - val_loss: 1.0287 - val_accuracy: 0.7084
Epoch 2/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.8159 - accuracy: 0.7792 - val_loss: 0.8540 - val_accuracy: 0.7603
Epoch 3/100
8000/8000 [==============================] - 38s 5ms/step - loss: 0.6638 - accuracy: 0.8159 - val_loss: 0.7048 - val_accuracy: 0.7984
Epoch 4/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.5734 - accuracy: 0.8331 - val_loss: 0.6382 - val_accuracy: 0.8114
Epoch 5/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.5234 - accuracy: 0.8470 - val_loss: 0.5952 - val_accuracy: 0.8253
Epoch 6/100
8000/8000 [==============================] - 35s 4ms/step - loss: 0.4865 - accuracy: 0.8570 - val_loss: 0.5593 - val_accuracy: 0.8380
Epoch 7/100
8000/8000 [==============================] - 35s 4ms/step - loss

8000/8000 [==============================] - 35s 4ms/step - loss: 0.0919 - accuracy: 0.9708 - val_loss: 0.5965 - val_accuracy: 0.8731
Epoch 57/100
8000/8000 [==============================] - 34s 4ms/step - loss: 0.0900 - accuracy: 0.9715 - val_loss: 0.6007 - val_accuracy: 0.8723
Epoch 58/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.0881 - accuracy: 0.9717 - val_loss: 0.6024 - val_accuracy: 0.8729
Epoch 59/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.0860 - accuracy: 0.9724 - val_loss: 0.6153 - val_accuracy: 0.8723
Epoch 60/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.0842 - accuracy: 0.9729 - val_loss: 0.6093 - val_accuracy: 0.8733
Epoch 61/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.0822 - accuracy: 0.9736 - val_loss: 0.6206 - val_accuracy: 0.8727
Epoch 62/100
8000/8000 [==============================] - 33s 4ms/step - loss: 0.0806 - accuracy: 0.9740 - val_loss: 0.6310 - val_a

In [29]:
from keras.models import load_model
model.save('language_translator.h5')